In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [10, 7]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
M55_path = 'C:\\Users\\marti\\Desktop\\school work\\Dartmouth\\GC_ages\\M55'

In [ ]:
AS_center = [3005.49976,2997.75391]
OBS_center = [2995.02393,3020.84351]
V_diff=30.98
I_diff = V_diff-0.737

In [ ]:
M55_fitstars = pd.read_csv("{}\\M55_fitstars.dat".format(M55_path),sep=',')

In [ ]:
def find_fit_val(arr,x):
    for idx in range(len(arr)):
        if x < arr[idx]:
            return arr[idx]
    return arr[-1]

In [ ]:
df_V_bound = pd.read_csv("{}\\inputfiles\\VerrBoundary.dat".format(M55_path),sep='\s+',skiprows=3, names=['Rad', 'Mag', 'Nstar', 'Completness'])
Rad_bounds = np.sort(np.unique(df_V_bound['Rad'].values))
V_mag_bounds = np.sort(np.unique(df_V_bound['Mag'].values))
rad = np.sqrt((M55_fitstars['x'].values - OBS_center[0])**2 + (M55_fitstars['y'].values - OBS_center[1])**2)

In [ ]:
V_bin = []
for i in range(len(M55_fitstars)):
    min_V=find_fit_val(V_mag_bounds,M55_fitstars['v'].values[i] - V_diff)
    min_r = find_fit_val(Rad_bounds,rad[i])
    for j in range(len(df_V_bound)):
        if (np.abs(df_V_bound['Rad'].values[j] - min_r) < 0.01) & (np.abs(df_V_bound['Mag'].values[j] - min_V) < 0.01):
            V_bin.append(j+1)
            break

In [ ]:
df_I_bound = pd.read_csv("{}\\inputfiles\\IerrBoundary.dat".format(M55_path),sep='\s+',skiprows=3, names=['Rad', 'Mag', 'Nstar', 'Completness'])
Rad_bounds = np.sort(np.unique(df_I_bound['Rad'].values))
I_mag_bounds = np.sort(np.unique(df_I_bound['Mag'].values))
rad = np.sqrt((M55_fitstars['x'].values - OBS_center[0])**2 + (M55_fitstars['y'].values - OBS_center[1])**2)

In [ ]:
I_bin = []
for i in range(len(M55_fitstars)):
    min_I = find_fit_val(I_mag_bounds,M55_fitstars['i'].values[i] - I_diff)
    min_r = find_fit_val(Rad_bounds,rad[i])
    for j in range(len(df_I_bound)):
        if (np.abs(df_I_bound['Rad'].values[j] - min_r) < 0.01) & (np.abs(df_I_bound['Mag'].values[j] - min_I) < 0.01):
            I_bin.append(j+1)
            break

In [ ]:
M55_fitstars['Vbin'] = V_bin
M55_fitstars['Ibin'] = I_bin

In [ ]:
M55_fitstars.to_csv("{}\\M55_fitstars_with_bins.dat".format(M55_path))